In [1]:
import sentiword as swn
net = swn.CustomSentiWordNet()
normalized = swn.normalizer()

c:\Users\Bayu Adjie Sidharta\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to C:\Users\Bayu Adjie
[nltk_data]     Sidharta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
df = pd.read_csv('cleaned.csv')
df

,username,tweetID,content,likeCount,retweetCount,quoteCount,replyCount,label,weighted_sentiment
0,banu_sukamto,1129304516304572416,"@AndiArief__ Bang, saya salah 1 pengagum @Part...",2,1,0,1,1.0,2.0
1,banu_sukamto,1123448831029534721,@FerdinandHaean2 @prabowo @sandiuno Hormat ku ...,4,1,0,2,1.0,3.0
2,banu_sukamto,1119565995209703424,@atoillahsmart @Fahrihamzah @Rozi_Handri @bpkr...,1,1,0,0,-1.0,-1.5
3,banu_sukamto,1117130509513977856,#4HariLagiMenujuAdilMakmur \n\nLet's VOTE : ✌✌...,1,1,0,0,0.0,0.0
4,banu_sukamto,1111951971617894401,@MichelAdamNew #PrabowoBentengNKRI \n👉 ✔✅ Bapa...,2,1,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
599995,indosatudotnet,1047003494216060928,"Dikabarkan Diculik &amp; Dipukuli, Anak Buah S...",0,0,0,0,0.0,0.0
599996,indosatudotnet,1045735188452364289,11 Poin Penjelasan Lengkap Dubes RI soal Duduk...,0,0,0,0,0.0,0.0
599997,indosatudotnet,1042782202335879168,Megawati Ketawa Ada Usul Debat Pilpres Pakai B...,4,1,0,2,0.0,0.0
599998,indosatudotnet,1042655086239838208,Pendiri Partai Gerindra Deklarasikan Relawan K...,3,2,0,0,0.0,0.0


In [3]:
print(df.iloc[0]['content'])
print(normalized.normalize(df.iloc[0]['content']))
net.calculate_sentiment(normalized.normalize(df.iloc[0]['content']))

@AndiArief__ Bang, saya salah 1 pengagum @PartaiDemokrat ,2 x pemilu Bapak @SBYudhoyono pilihan saya. Saat ini saya pilih 02 karena terdiri dari Partai pilihan saya. @Partaiberkarya @PartaiDemokrat @Gerindra @PartaiPAN @PKSejahtera
Untuk apa cari kambing hitam, kalau ada problem di internal✌


([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [3]:
zipped = zip(df['tweetID'], df['content'])
# zipped = zip(df.head(1005)['tweetID'], df.head(1005)['content'])


In [4]:
# for x in zipped:
#     print(x)

In [1]:
import concurrent.futures
import time
from datetime import datetime

def get_sentiment(zipped):
    
    global total_runs 
    total_runs = total_runs + 1

    if total_runs % 1000 == 0:
        print('Run: {}'.format(total_runs))
    tweetID, text = zipped
    pos, neg = net.calculate_sentiment(normalized.normalize(text))
    
    return {
        'tweetID': tweetID,
        'pos_sentiword': sum(pos),
        'neg_sentiword': sum(neg),
        'len': len(pos)
    }


with concurrent.futures.ThreadPoolExecutor() as executor:
    start = datetime.now()
    res = {}
    total_runs = 0
    futures =  [executor.submit(get_sentiment, x) for x in zipped]
    for futures in concurrent.futures.as_completed(futures):

        result = futures.result()
        res[result['tweetID']] = {
            'pos_sentiword': result['pos_sentiword'],
            'neg_sentiword': result['neg_sentiword'],
            'len': result['len']
        }

    end = datetime.now()
    print('Time taken: {}'.format(end - start))



NameError: name 'zipped' is not defined

In [6]:
df_sentiword = pd.DataFrame.from_dict(res, orient='index')
df_sentiword.index.name = 'tweetID'
df_sentiword = df_sentiword.reset_index()
df_sentiword

,tweetID,pos_sentiword,neg_sentiword,len
0,1067038496748199938,0.625,0.750,6
1,1052955028506341376,0.000,0.250,3
2,1051087927025917952,0.000,0.000,2
3,1079592016051240961,0.875,0.000,9
4,1041211511081906177,0.000,0.000,4
...,...,...,...,...
599995,1040390367424593920,0.000,0.000,0
599996,1131116233728843776,0.250,0.500,10
599997,1117037232982908928,0.250,0.125,4
599998,1120777817250091008,0.000,0.000,3


In [10]:
df_joined = df.merge(df_sentiword, on='tweetID')
df_joined

,username,tweetID,content,likeCount,retweetCount,quoteCount,replyCount,label,weighted_sentiment,pos_sentiword,neg_sentiword,len
0,banu_sukamto,1129304516304572416,"@AndiArief__ Bang, saya salah 1 pengagum @Part...",2,1,0,1,1.0,2.0,0.000,0.125,9
1,banu_sukamto,1123448831029534721,@FerdinandHaean2 @prabowo @sandiuno Hormat ku ...,4,1,0,2,1.0,3.0,0.500,0.000,9
2,banu_sukamto,1119565995209703424,@atoillahsmart @Fahrihamzah @Rozi_Handri @bpkr...,1,1,0,0,-1.0,-1.5,0.000,0.500,4
3,banu_sukamto,1117130509513977856,#4HariLagiMenujuAdilMakmur \n\nLet's VOTE : ✌✌...,1,1,0,0,0.0,0.0,0.000,0.000,2
4,banu_sukamto,1111951971617894401,@MichelAdamNew #PrabowoBentengNKRI \n👉 ✔✅ Bapa...,2,1,0,0,0.0,0.0,0.125,0.000,4
...,...,...,...,...,...,...,...,...,...,...,...,...
599995,indosatudotnet,1047003494216060928,"Dikabarkan Diculik &amp; Dipukuli, Anak Buah S...",0,0,0,0,0.0,0.0,0.000,0.125,3
599996,indosatudotnet,1045735188452364289,11 Poin Penjelasan Lengkap Dubes RI soal Duduk...,0,0,0,0,0.0,0.0,1.250,0.000,9
599997,indosatudotnet,1042782202335879168,Megawati Ketawa Ada Usul Debat Pilpres Pakai B...,4,1,0,2,0.0,0.0,0.250,0.375,7
599998,indosatudotnet,1042655086239838208,Pendiri Partai Gerindra Deklarasikan Relawan K...,3,2,0,0,0.0,0.0,0.625,0.375,8


In [2]:
print(df_joined.head(1))

NameError: name 'df_joined' is not defined

In [8]:
df_joined.isna().sum()


username              0
tweetID               0
content               0
likeCount             0
retweetCount          0
quoteCount            0
replyCount            0
label                 0
weighted_sentiment    0
pos_sentiword         0
neg_sentiword         0
len                   0
dtype: int64

In [9]:
# df_joined.to_csv('sentiword.csv')

In [16]:
not_found = net.get_not_found()
not_found

{'x': 2750,
 'pemilu': 8152,
 'internal': 336,
 'ku': 14713,
 'tv': 2150,
 'prosentasenya': 11,
 's': 5164,
 'vote': 450,
 'indonesia': 28669,
 'election': 295,
 'of': 5002,
 'president': 553,
 'members': 26,
 'house': 138,
 'representatives': 9,
 'will': 801,
 'be': 1498,
 'held': 27,
 'together': 110,
 'on': 2566,
 'people': 1442,
 'very': 318,
 'enthusiastic': 2,
 'to': 6421,
 'welcome': 235,
 'this': 2255,
 'historical': 7,
 'day': 965,
 'hgu': 303,
 'lahan': 1047,
 'berpikiran': 66,
 'capres': 4442,
 'sby': 1973,
 'riau': 561,
 'please': 622,
 'pikir': 1160,
 'survey': 1075,
 'nagami': 1,
 'httpurl': 281357,
 'hajarrr': 7,
 'gebuk': 70,
 'tweet': 1017,
 'ketulungan': 52,
 'guoblog': 6,
 'laah': 91,
 'beneran': 1250,
 'siakan': 53,
 'kedzoliman': 64,
 'do': 1668,
 'selemah': 11,
 'berdo': 135,
 'nenteng': 9,
 'apan': 8,
 'razia': 149,
 'aceh': 1210,
 'jakarta': 5863,
 'gini': 4324,
 'nembakin': 19,
 'layani': 75,
 'tembakin': 11,
 'all': 1200,
 'out': 708,
 'element': 14,
 'miliki'

In [21]:
df_not_found = pd.DataFrame.from_dict(not_found, orient='index', columns=['count'])
df_not_found.index.name = 'Word'
df_not_found = df_not_found.reset_index()
df_not_found

,Word,count
0,x,2750
1,pemilu,8152
2,internal,336
3,ku,14713
4,tv,2150
...,...,...
180952,situbodo,1
180953,majlebbbb,1
180954,moratelindo,1
180955,yeobo,1


In [22]:
df_not_found.sort_values(by='count', ascending=False).head(20)

,Word,count
38,httpurl,281357
8,indonesia,28669
148,jokowi,27317
106,prabowo,16676
3,ku,14713
1242,allah,14684
367,isu bohong,8163
1,pemilu,8152
23,to,6421
3806,aamiin,6268


In [23]:
# df_not_found.to_csv('not_found.csv')

In [24]:
len(not_found)

180957